In [1]:
import pandas as pd
import numpy as np
from keras.utils import *

X_train = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt", header=None, sep="\s+")
y_train = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt", header=None)

X_test = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/test/X_test.txt", header=None, sep="\s+")
y_test = pd.read_csv("../Dataset/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt", header=None)

X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((7352, 1, 561), (7352, 7), (2947, 1, 561), (2947, 7))

In [2]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *

def build_model(hp):
    modelo = Sequential()
    modelo.add(Bidirectional(LSTM(hp.Int("units_lstm_1", min_value=1, max_value=600), return_sequences=True)))
    modelo.add(Dropout(hp.Float("dropout_1", min_value=0.001, max_value=0.5)))
    modelo.add(Bidirectional(LSTM(hp.Int("units_lstm_2", min_value=1, max_value=600))))
    modelo.add(Dropout(hp.Float("dropout_2", min_value=0.001, max_value=0.5)))
    modelo.add(Dense(7, activation="sigmoid"))
    modelo.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=hp.Choice("learning_rate", [0.0001, 0.001, 0.01])), metrics=["accuracy"])
    return modelo

In [3]:
from keras_tuner import *
from keras.callbacks import *

tuner = BayesianOptimization(hypermodel=build_model, objective='accuracy')
tuner.search(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=10, callbacks=[EarlyStopping(patience=2)])

Trial 10 Complete [01h 04m 32s]
accuracy: 0.7660500407218933

Best accuracy So Far: 0.97633296251297
Total elapsed time: 2d 17h 29m 35s


In [4]:
hyperparameters = tuner.get_best_hyperparameters()[0]
print(f"Units LSTM 1: {hyperparameters.get('units_lstm_1')}")
print(f"Dropout 1: {hyperparameters.get('dropout_1')}")
print(f"Units LSTM 2: {hyperparameters.get('units_lstm_2')}")
print(f"Dropout 2: {hyperparameters.get('dropout_2')}")
print(f"Learning Rate: {hyperparameters.get('learning_rate')}")

Units LSTM 1: 92
Dropout 1: 0.3227008199881794
Units LSTM 2: 18
Dropout 2: 0.4575568770239258
Learning Rate: 0.0001


In [15]:
best_model = tuner.get_best_models()[0]
best_model.build(input_shape=(7352, 1, 561))
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (7352, 1, 184)           481344    
 l)                                                              
                                                                 
 dropout (Dropout)           (7352, 1, 184)            0         
                                                                 
 bidirectional_1 (Bidirectio  (7352, 36)               29232     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (7352, 36)                0         
                                                                 
 dense (Dense)               (7352, 7)                 259       
                                                                 
Total params: 510,835
Trainable params: 510,835
Non-trai